<a href="https://colab.research.google.com/github/vishal7379/Colab/blob/main/NL_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faker

import random
import os
import torch
import torch.nn as nn
from faker import Faker
from torch.utils.data import Dataset, DataLoader
from collections import Counter

fake = Faker()

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.0 MB/s eta 0:00:00
Device: cuda


In [6]:
SQL_PATTERNS = (
    ["join"] * 5 +
    ["join_where"] * 4 +        # ⭐ NEW
    ["group_by"] * 3 +
    ["having"] * 2 +
    ["where"] * 2 +
    ["aggregation"] * 2 +
    ["order_by"] * 3 +         # ⭐ NEW
    ["limit"] * 2 +            # ⭐ NEW
    ["order_by_limit"] * 3 +   # ⭐ VERY IMPORTANT
    ["multi_select"] +
    ["simple_select"]
)


In [7]:
BASE_COLUMNS = [
    "name","email","age","salary",
    "department","city","country",
    "price","amount","quantity",
    "created_at","updated_at"
]

FK_COLUMNS = [
    "user_id",
    "order_id",
    "product_id",
    "customer_id"
]

In [8]:
COLUMN_TYPES = {
    "numeric":[
        "age","salary","price",
        "amount","quantity"
    ],
    "text":[
        "name","email",
        "department","city","country"
    ],
    "date":[
        "created_at","updated_at"
    ]
}

In [9]:
def generate_relational_schema():

    num_tables = random.randint(2,5)

    fake.unique.clear()

    tables = {}
    relationships = {}

    table_names = [fake.unique.word() for _ in range(num_tables)]

    for t in table_names:

        cols = {
            "numeric": random.sample(COLUMN_TYPES["numeric"],
                                     random.randint(1,3)),

            "text": random.sample(COLUMN_TYPES["text"],
                                  random.randint(1,3)),

            "date": random.sample(COLUMN_TYPES["date"],
                                  random.randint(0,1))
        }

        # flatten
        flat_cols = ["id"]

        for v in cols.values():
            flat_cols.extend(v)

        tables[t] = {
            "all": flat_cols,
            "numeric": cols["numeric"],
            "text": cols["text"],
            "date": cols["date"]
        }

    relationships_list = []

    for i in range(1, num_tables):

        parent = table_names[i-1]
        child = table_names[i]

        fk = f"{parent}_id"

        tables[child]["all"].append(fk)

        relationships_list.append(
            (child, fk, parent, "id")
        )

    return {
        "tables": tables,
        "relationships": relationships_list
    }

In [10]:
def schema_to_text(schema):

    parts = []

    for table, col_dict in schema["tables"].items():

        cols = col_dict["all"]

        col_tokens = " ".join([f"[COL] {c}" for c in cols])
        parts.append(f"[TABLE] {table} {col_tokens}")

    # relationships
    for child, fk, parent, pk in schema["relationships"]:
        parts.append(
            f"[REL] {child}.{fk} -> {parent}.{pk}"
        )

    return " ".join(parts)

In [11]:
SELECT_TEMPLATES = [
    "show {col} from {table}",
    "list {col} in {table}",
    "display {col} from {table}",
    "what are the {col} in {table}",
]

In [12]:
def generate_sql(schema):

    pattern = random.choice(SQL_PATTERNS)

    tables = schema["tables"]
    rels = schema["relationships"]

    table = random.choice(list(tables.keys()))

    columns = tables[table]["all"]
    numeric_cols = tables[table]["numeric"]
    text_cols = tables[table]["text"]

    # ---------- SIMPLE SELECT ----------
    if pattern == "simple_select":

        selected = random.sample(
            columns,
            random.randint(1, min(3, len(columns)))
        )

        question = f"show {', '.join(selected)} from {table}"
        sql = f"SELECT {', '.join(selected)} FROM {table}"


    # ---------- MULTI SELECT ----------
    elif pattern == "multi_select":

        selected = random.sample(
            columns,
            random.randint(2, min(4, len(columns)))
        )

        question = f"show {', '.join(selected)} from {table}"
        sql = f"SELECT {', '.join(selected)} FROM {table}"


    # ---------- WHERE ----------
    elif pattern == "where":

        # Prefer numeric filters
        if numeric_cols and random.random() < 0.7:

            col = random.choice(numeric_cols)
            operator = random.choice(["=", ">", "<"])
            value = random.randint(1, 100)

        else:

            col = random.choice(text_cols if text_cols else columns)
            operator = "="
            value = f"'{fake.word()}'"

        question = f"show {col} from {table} where {col} {operator} {value}"
        sql = f"SELECT {col} FROM {table} WHERE {col} {operator} {value}"


    # ---------- AGGREGATION ----------
    elif pattern == "aggregation":

        agg = random.choice(["COUNT","SUM","AVG","MIN","MAX"])

        if agg in ["SUM","AVG"]:

            if not numeric_cols:
                return generate_sql(schema)

            col = random.choice(numeric_cols)

        elif agg in ["MIN","MAX"]:

            candidates = numeric_cols + text_cols
            if not candidates:
                return generate_sql(schema)

            col = random.choice(candidates)

        else:  # COUNT
            col = random.choice(columns)

        question = f"what is the {agg.lower()} of {col} in {table}"
        sql = f"SELECT {agg}({col}) FROM {table}"
        # ---------- ORDER BY ----------
    elif pattern == "order_by":

        # prefer numeric columns for sorting
        if numeric_cols:
            col = random.choice(numeric_cols)
        else:
            col = random.choice(columns)

        direction = random.choice(["ASC","DESC"])

        question = f"show all records from {table} ordered by {col} {direction.lower()}"

        sql = f"""
        SELECT *
        FROM {table}
        ORDER BY {col} {direction}
        """
        # ---------- LIMIT ----------
    elif pattern == "limit":

        limit_val = random.randint(3,20)

        question = f"show first {limit_val} rows from {table}"

        sql = f"""
SELECT *
FROM {table}
LIMIT {limit_val}
"""
        # ---------- ORDER BY + LIMIT ----------
    elif pattern == "order_by_limit":

        if numeric_cols:
            col = random.choice(numeric_cols)
        else:
            col = random.choice(columns)

        direction = random.choice(["ASC","DESC"])
        limit_val = random.randint(3,15)

        question = f"show top {limit_val} records from {table} ordered by {col}"

        sql = f"""
SELECT *
FROM {table}
ORDER BY {col} {direction}
LIMIT {limit_val}
"""


    # ---------- GROUP BY ----------
    elif pattern == "group_by":

        group_candidates = text_cols if text_cols else columns
        group_col = random.choice(group_candidates)

        if numeric_cols:
            agg_col = random.choice(numeric_cols)
        else:
            return generate_sql(schema)

        agg = random.choice(["COUNT","SUM","AVG","MIN","MAX"])

        question = f"{agg.lower()} of {agg_col} grouped by {group_col}"

        sql = f"""
SELECT {group_col}, {agg}({agg_col})
FROM {table}
GROUP BY {group_col}
"""


    # ---------- HAVING ----------
    elif pattern == "having":

        group_candidates = text_cols if text_cols else columns
        group_col = random.choice(group_candidates)

        if not numeric_cols:
            return generate_sql(schema)

        agg_col = random.choice(numeric_cols)
        agg = random.choice(["COUNT","SUM","AVG","MIN","MAX"])

        operator = random.choice([">", "<", "="])
        value = random.randint(1,50)

        question = f"{agg.lower()} of {agg_col} grouped by {group_col} having value {operator} {value}"

        sql = f"""
SELECT {group_col}, {agg}({agg_col})
FROM {table}
GROUP BY {group_col}
HAVING {agg}({agg_col}) {operator} {value}
"""
    # ---------- JOIN + WHERE ----------
    elif pattern == "join_where" and rels:

        child, fk, parent, pk = random.choice(rels)

        if random.random() < 0.5:
            left, right = parent, child
            left_key, right_key = pk, fk
        else:
            left, right = child, parent
            left_key, right_key = fk, pk

        right_numeric = tables[right]["numeric"]
        right_text = tables[right]["text"]

        if right_numeric and random.random() < 0.7:

            where_col = random.choice(right_numeric)
            operator = random.choice([">","<","="])
            value = random.randint(1,100)

        else:

            candidates = right_text if right_text else right_numeric
            if not candidates:
                return generate_sql(schema)

            where_col = random.choice(candidates)
            operator = "="
            value = f"'{fake.word()}'"

        select_col = random.choice(tables[left]["all"])

        question = f"show {select_col} from {left} joined with {right} where {where_col} {operator} {value}"

        sql = f"""
SELECT {left}.{select_col}
FROM {left}
JOIN {right}
ON {left}.{left_key} = {right}.{right_key}
WHERE {right}.{where_col} {operator} {value}
"""

    # ---------- JOIN ----------
    elif pattern == "join" and rels:

        child, fk, parent, pk = random.choice(rels)

        if random.random() < 0.5:
            left, right = parent, child
            left_key, right_key = pk, fk
        else:
            left, right = child, parent
            left_key, right_key = fk, pk

        select_col = random.choice(tables[left]["all"])

        question = f"show {select_col} from {left} joined with {right}"

        sql = f"""
SELECT {left}.{select_col}
FROM {left}
JOIN {right}
ON {left}.{left_key} = {right}.{right_key}
"""

    else:
        return generate_sql(schema)

    return question.strip(), sql.strip()

In [13]:
def build_dataset_by_schema(num_schemas=8000,
                            queries_per_schema=8):

    schemas = [generate_relational_schema()
               for _ in range(num_schemas)]

    split = int(0.9 * num_schemas)

    train_s = schemas[:split]
    test_s = schemas[split:]

    def build_examples(schema_list):

        data = []

        for schema in schema_list:

            schema_text = schema_to_text(schema)

            for _ in range(queries_per_schema):

                q, sql = generate_sql(schema)

                model_input = f"""
Schema:
{schema_text}

Question:
{q}
"""

                data.append({
                    "input": model_input.strip(),
                    "output": sql.strip()
                })

        return data

    return build_examples(train_s), build_examples(test_s)


train, test = build_dataset_by_schema()

print("Train:", len(train))
print("Test:", len(test))

Train: 57600
Test: 6400


In [14]:
import json

with open("train.json", "w") as f:
    json.dump(train, f)

with open("test.json", "w") as f:
    json.dump(test, f)

print("✅ Dataset saved as JSON")


✅ Dataset saved as JSON


In [15]:
def build_vocab(data):

    counter = Counter()

    for row in data:
        counter.update(row["input"].split())
        counter.update(row["output"].split())

    vocab = {w:i+2 for i,(w,_) in enumerate(counter.items())}
    vocab["<pad>"] = 0
    vocab["<unk>"] = 1

    return vocab

vocab = build_vocab(train)

torch.save(vocab, "vocab.pt")

In [17]:
MAX_LEN = 220

def encode(text):

    tokens = text.split()

    ids = [vocab.get(t,1) for t in tokens][:MAX_LEN]

    ids += [0]*(MAX_LEN-len(ids))

    return torch.tensor(ids)

class NL2SQLDataset(Dataset):

    def __init__(self,data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):

        row = self.data[idx]

        return encode(row["input"]), encode(row["output"])

In [ ]:
import json

with open("train.json") as f:
    train = json.load(f)

with open("test.json") as f:
    test = json.load(f)

print("✅ Dataset loaded!")
print("Train size:", len(train))


In [18]:
class NL2SQLModel(nn.Module):

    def __init__(self, vocab_size, d_model=256):
        super().__init__()

        self.embed = nn.Embedding(vocab_size, d_model)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=8,
            num_encoder_layers=3,
            num_decoder_layers=3,
            batch_first=True
        )

        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):

        src_mask = (src == 0)
        tgt_mask = (tgt == 0)

        src = self.embed(src)
        tgt = self.embed(tgt)

        out = self.transformer(
            src,
            tgt,
            src_key_padding_mask=src_mask,
            tgt_key_padding_mask=tgt_mask
        )

        return self.fc(out)

In [19]:
for i in range(20):
    print(train[i]["input"])
    print(train[i]["output"])
    print("-----")

Schema:
[TABLE] new [COL] id [COL] price [COL] email [TABLE] hospital [COL] id [COL] age [COL] quantity [COL] price [COL] name [COL] country [COL] new_id [REL] hospital.new_id -> new.id

Question:
what is the count of quantity in hospital
SELECT COUNT(quantity) FROM hospital
-----
Schema:
[TABLE] new [COL] id [COL] price [COL] email [TABLE] hospital [COL] id [COL] age [COL] quantity [COL] price [COL] name [COL] country [COL] new_id [REL] hospital.new_id -> new.id

Question:
show price from hospital where price = 12
SELECT price FROM hospital WHERE price = 12
-----
Schema:
[TABLE] new [COL] id [COL] price [COL] email [TABLE] hospital [COL] id [COL] age [COL] quantity [COL] price [COL] name [COL] country [COL] new_id [REL] hospital.new_id -> new.id

Question:
show country from hospital joined with new
SELECT hospital.country
FROM hospital
JOIN new
ON hospital.new_id = new.id
-----
Schema:
[TABLE] new [COL] id [COL] price [COL] email [TABLE] hospital [COL] id [COL] age [COL] quantity [COL

In [20]:
train_loader = DataLoader(
    NL2SQLDataset(train),
    batch_size=32,
    shuffle=True
)

model = NL2SQLModel(len(vocab)).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=0)

In [21]:
CHECKPOINT_PATH = "nl2sql_checkpoint.pt"

def save_checkpoint(epoch, model, optimizer, loss):

    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "loss": loss
    }, CHECKPOINT_PATH)

def load_checkpoint(model, optimizer):

    if os.path.exists(CHECKPOINT_PATH):

        ckpt = torch.load(CHECKPOINT_PATH,
                          map_location=device)

        model.load_state_dict(ckpt["model_state"])
        optimizer.load_state_dict(ckpt["optimizer_state"])

        print("Resuming from epoch:",
              ckpt["epoch"]+1)

        return ckpt["epoch"] + 1

    return 0

In [22]:
EPOCHS = 20
best_loss = float("inf")

start_epoch = load_checkpoint(model, optimizer)

for epoch in range(start_epoch, EPOCHS):

    model.train()
    total_loss = 0

    for x,y in train_loader:

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        output = model(x, y[:,:-1])

        loss = loss_fn(
            output.reshape(-1, len(vocab)),
            y[:,1:].reshape(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    print(f"Epoch {epoch} | Loss {avg_loss}")

    save_checkpoint(epoch, model, optimizer, avg_loss)

    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(),
                   "best_model.pt")
        print("⭐ BEST MODEL SAVED")

KeyboardInterrupt: 

In [ ]:
# ---------- LOAD VOCAB ----------
vocab = torch.load("vocab.pt")
inv_vocab = {i:w for w,i in vocab.items()}

device = "cuda" if torch.cuda.is_available() else "cpu"


# ---------- REBUILD MODEL ----------
model = NL2SQLModel(len(vocab)).to(device)

model.load_state_dict(
    torch.load("best_model.pt", map_location=device)
)

model.eval()

print("✅ Best model loaded!")


In [ ]:
MAX_LEN = 220

def encode(text):

    tokens = text.split()

    ids = [vocab.get(t,1) for t in tokens][:MAX_LEN]

    ids += [0]*(MAX_LEN-len(ids))

    return torch.tensor(ids)


In [ ]:
def generate_query(model, text, max_len=220):

    model.eval()

    src = encode(text).unsqueeze(0).to(device)

    tgt = torch.zeros((1,1), dtype=torch.long).to(device)

    for _ in range(max_len):

        tgt_mask = nn.Transformer.generate_square_subsequent_mask(
            tgt.size(1)
        ).to(device)

        with torch.no_grad():

            output = model.transformer(
                model.embed(src),
                model.embed(tgt),
                tgt_mask=tgt_mask,
                src_key_padding_mask=(src==0),
                tgt_key_padding_mask=(tgt==0)
            )

            logits = model.fc(output)

        next_token = logits.argmax(-1)[:,-1].unsqueeze(0)

        tgt = torch.cat([tgt, next_token], dim=1)

        if next_token.item() == 0:
            break


    tokens = [
        inv_vocab.get(i,"")
        for i in tgt.squeeze().tolist()
    ]

    return " ".join(tokens)
